In [1]:
# https://huggingface.co/IlyaGusev/mbart_ru_sum_gazeta

In [2]:
#%pip install git+https://github.com/huggingface/transformers.git
#%pip install ipywidgets

In [3]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [4]:
import torch
torch.cuda.is_available()

True

In [5]:
model_name = "IlyaGusev/mbart_ru_sum_gazeta"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

In [6]:
def summarize(article_text, tokenizer, model, device):    
    input_ids = tokenizer.prepare_seq2seq_batch(
        [article_text],
        src_lang="en_XX", # fairseq training artifact
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=600
    ).to(device)["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=162,
        no_repeat_ngram_size=3,
        num_beams=10, # 5
        top_k=0
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return summary

In [7]:
with open('data/content_a.txt') as file:
    article_text = file.read()
article_text = article_text.replace('\n','. ')
print(article_text)

 До этого момента Брайан был удивлен и до крайности  озадачен.  Теперь. он начал испытывать страх. Да, он был по-настоящему испуган. До сих пор не. было времени испугаться. Хотелось, чтобы так и  продолжалось.  Теперь  это. стало невозможным. Он  переключил  радио  на  аварийную  волну,  попытался. снова. Никакого результата. Все равно что в Манхэттене набрать по телефону. 911 и услышать автоответчик, сообщающий, что все разъехались  на  уик-энд.. Когда просишь помощи на аварийной волне, всегда получишь нужный ответ..      "По крайней мере, до сих пор", - подумал Брайан..      Включил УНИКОМ, по которому пилоты частных самолетов  получали  совет. для  посадки  на  ближайших   маленьких   полосах.   Полная   тишина.   Он. вслушивался...   и  вообще  ни  звука  не  слышал.  Поверить  в  это  было. невозможно. Пилоты-частники всегда  болтали  между  собой,  как  грачи  на. телефонных проводах. Девушка с Пайпера интересовалась  погодой.  Парень  с. Чессны немедленно отбросит копыта, если к

In [8]:
%time print('==>', summarize(article_text, tokenizer, model, device))

/home/alex/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3255: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  warnings.warn(


==> В аэропорту Лос-Анджелеса приземлился американский пилот частного самолета Ник Брайан. Он переключил радио на аварийную волну, попытался связаться с диспетчерами, но так и не смог.
CPU times: user 1.65 s, sys: 23.4 ms, total: 1.67 s
Wall time: 1.7 s
